In [1]:
#!pip install simpletransformers
#!pip install transformers ==3.0.2 
#!pip install torch
#Loading libraries
import torch
from simpletransformers.t5 import T5Model
import pandas as pd

In [2]:
#Loading trainiing data 
#This has this will have 5 columns 1) annotation, 2) student_note , 3) teacher_feedback 4) phrases  & 5) score  
#score column is for reference and it is not used for BERT or T5 trainining
# teacher_feedback column is used for T5 model traiining while phrases column is used for BERT model
df_train = pd.read_csv('train_data.csv')
prefix = 'x: '
df_train['prefix'] = prefix
df_train['input_text'] = 'Annotation: ' + df_train['annotation'] + 'Note: '+ df_train['student_note']
df_train['target_text'] = df_train['teacher_feedback']
df_train1 = df_train[['prefix', 'input_text', 'target_text']]
df_train1.shape
#Data for testing. same columns as df_train
df_test = pd.read_csv('test_data.csv') 

In [5]:
#Defining T5 model
model = T5Model( "t5","t5-base")
model_args = {
    "max_seq_length": 256,
    "train_batch_size": 16,
    "eval_batch_size": 64,
    "num_train_epochs": 3,
    #"evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    
    "use_multiprocessing": False,
    "fp16": False,

    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,

    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'n_gpu': 2,
    'process_count': 25,
    'model_name':"t5-base",
    'truncation': False,
    "model_max_length": 512,
}


In [6]:
#Fine-tuning (Training) T5 model
model.train_model (df_train1,  args=model_args)


  0%|          | 0/40324 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/2521 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Running Epoch 1 of 3:   0%|          | 0/2521 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/2521 [00:00<?, ?it/s]

(7563, 2.601373496894781)

In [7]:
#Saving T5 model to disk
model.model.save_pretrained('./t5_feedback_gen')
model.tokenizer.save_pretrained('./t5_feedback_gen')

('./t5_feedback_gen/tokenizer_config.json',
 './t5_feedback_gen/special_tokens_map.json',
 './t5_feedback_gen/spiece.model',
 './t5_feedback_gen/added_tokens.json')

In [8]:
#Loading T5 model for inference
model = T5Model("t5", './t5_feedback_gen')


In [9]:
annotation = "night was dark. The wind blew." 
student_note = "there are 13 stanzas "
prefix = 'x: '
text =  prefix + 'Annotation: ' + annotation + 'Note: ' + student_note
print("Feedback Generated From T5: ", model.predict([text]) )



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Feedback Generated From T5:  ['What kinds of poetic elements and figurative language do you see?']


In [10]:
annotation = "banks had very few rules about loaning out money." 
student_note = "Seems like the banks should have done something sooner"
prefix = 'x: '
text =  prefix + 'Annotation: ' + annotation + 'Note: ' + student_note
print("Feedback Generated From T5: ", model.predict([text]) )


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Feedback Generated From T5:  ['Yes, this is why banks are regulated now.']


In [11]:
annotation = "He was heart-broken." 
student_note = "She is so caught up in trying to fit in she never realizes how she is hurting her husband."
prefix = 'x: '
text =  prefix + 'Annotation: ' + annotation + 'Note: ' + student_note
print("Feedback Generated From T5: ", model.predict([text]) )

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Feedback Generated From T5:  ['Yes, she is.']
